In [1]:
%matplotlib inline
import pandas as pd
import xgboost as xgb

In [2]:
from sklearn.datasets import fetch_covtype

forest_tree = fetch_covtype()

forest_tree_data = pd.DataFrame(forest_tree.data)
forest_tree_target = pd.Series(forest_tree.target)

# targetが1〜7なので、0〜6に補正する
forest_tree_target = forest_tree_target - 1

#print(forest_tree.DESCR)

In [3]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(forest_tree_data, forest_tree_target, test_size=0.5, shuffle=True)

In [4]:
params = {'eta': [1.0], 'n_estimators': [100], 'max_depth':[6], 'nthread': [2] }

In [5]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[CV] eta=1.0, max_depth=6, n_estimators=100, nthread=2 ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  eta=1.0, max_depth=6, n_estimators=100, nthread=2, score=0.897, total= 8.9min
[CV] eta=1.0, max_depth=6, n_estimators=100, nthread=2 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.9min remaining:    0.0s


[CV]  eta=1.0, max_depth=6, n_estimators=100, nthread=2, score=0.900, total= 9.1min
[CV] eta=1.0, max_depth=6, n_estimators=100, nthread=2 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 17.9min remaining:    0.0s


[CV]  eta=1.0, max_depth=6, n_estimators=100, nthread=2, score=0.899, total=10.2min
[CV] eta=1.0, max_depth=6, n_estimators=100, nthread=2 ...............
[CV]  eta=1.0, max_depth=6, n_estimators=100, nthread=2, score=0.894, total=11.6min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 39.7min finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=Fa

In [6]:
#print(clf.cv_results_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.898 (+/-0.004) for {'eta': 1.0, 'max_depth': 6, 'n_estimators': 100, 'nthread': 2}


In [7]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.8976
{'eta': 1.0, 'max_depth': 6, 'n_estimators': 100, 'nthread': 2}


In [8]:
model = clf.best_estimator_
#model.fit(train_x, train_y)

In [9]:
from sklearn.metrics import accuracy_score

pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9049
